In [ ]:
# default_exp main
from nbdev import *
from utilities.ipynb_docgen import *
!date

Mon May 10 17:20:02 PDT 2021


# wtlike interface
> Top-level interface to the rest of the package

In [ ]:
# export
from wtlike.bayesian import get_bb_partition
from wtlike.lightcurve import fit_cells, LightCurve, flux_plot
from wtlike.cell_data import partition_cells

class WtLike(LightCurve):
    """
    Inherit from `LightCurve` and add Bayesian Block capability
    """
    def bb_view(self, key=None, clear=False, **kwargs):
        """Return a view with the BB analysis applied
        
        Its `plot` function will by default show an overplot on the parent's data points.
        """
        #  a new instance 
        r = self.view()
        
        # bb analysis on this to make new  set of cells and poisson fits
        bb_edges  = get_bb_partition(self.config, self.fits,  key=key, clear=clear) 
        r.cells = partition_cells(self.config, self.cells, bb_edges)
        
        r.fits = fit_cells(self.config, r.cells, )
        r.isBB = True
        return r
    
    def plot(self, *pars, **kwargs):
        
        if getattr(self, 'isBB',  None) is None:
            return super().plot(*pars, **kwargs)
        else:
            return self.plot_BB(*pars, **kwargs)
            
    def plot_BB(self, ax=None, **kwargs):
        """Plot the light curve with BB overplot
        """
        import matplotlib.pyplot as plt

        figsize = kwargs.pop('figsize', (12,4))
        fignum = kwargs.pop('fignum', 1)
        ts_min = kwargs.pop('ts_min',-1)
        source_name =kwargs.pop('source_name', self.source_name)
        fig, ax = ig, ax = plt.subplots(figsize=figsize, num=fignum) if ax is None else (ax.figure, ax)


        colors = kwargs.pop('colors', ('lightblue', 'wheat', 'blue') )
        flux_plot(self.config, self.parent.fits, ax=ax, colors=colors, source_name=source_name, 
                  label=self.step_name+' bins', **kwargs)
        flux_plot(self.config, self.fits, ax=ax, step=True, 
                  label='BB overlay', zorder=10,**kwargs)

        fig.set_facecolor('white')
        return fig    


In [ ]:
show_doc(WtLike)

<h2 id="WtLike" class="doc_header"><code>class</code> <code>WtLike</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>WtLike</code>(**\*`pars`**, **\*\*`kwargs`**) :: [`LightCurve`](/wtlikelightcurve#LightCurve)

Inherit from [`LightCurve`](/wtlikelightcurve#LightCurve) and add Bayesian Block capability

In [ ]:
#collapse_input
full = None
# code for the demo--this shouid be collapsed
def demo():
    """
    ## Test/Demonstration with 3C 279
    
    > Note that this also demonstrates using `nbdoc` to have a single Jupyterlab cell generate a document
    
    First, the weekly light curve:
    {out1}    {fig1}
    
    Replot the figure with an expanded scale to see position of a flare:
    {fig2}
    {out3}    {fig3}
    {out4}    {fig4}
    
    Table of BB fits
    {bb_table}
    
    This can be compared with Figure 4 from the [Kerr paper](https://arxiv.org/pdf/1910.00140.pdf)
    {kerr_fig4}
    """
    global full
    
    #with capture_print('Create full weekly light curve') as out1:
    full = WtLike('3C 279')
    
    fig1 = figure(
        full.plot(yscale='log', ylim=(0.2,20), ts_min=4, figsize=(15,5), xlabel='MJD', fmt='.', fignum=1),
        caption='Full scale', width=600)  
    
    fig2 = figure(
        full.plot(  figsize=(15,5), xlabel='MJD', fmt='o', fignum=2,  xlim=(57100, 57300),),
        width=600)  
    
    with capture_print('Define orbit-based subset around large flare at MJD 57189') as out3:
        orbit = full.view((57186, 57191, 0))
    fig3 = figure(
        orbit.plot(fmt='o', tzero=57186,  ts_min=4,   fignum=3 ),
            width=600)
    
    with capture_print('Apply BB and overplot it with the cells on which it is based') as out4:
        bborbit = orbit.bb_view()
    fig4 = figure(
        bborbit.plot(fmt='o', tzero=57186,  ts_min=4,   fignum=4),
                width=600)
    
    bb_table = orbit.fluxes
    
    kerr_fig4 = image('kerr_fig4.png', width=600, caption=None)
    return locals()

nbdoc(demo)

SourceData: photons and exposure for 3C 279: Restoring from cache with key "3C 279_data"
WtLike: Source 3C 279 with:
	 data:       179,695  photons   from 2008-08-04 to 2021-04-21
	 exposure: 2,817,749 intervals  from 2008-08-04 to 2021-04-20
	 est. rates from weights: source 4.03e-07/s, background 3.25e-07/s, S/N ratio 1.24
CellData: Bin photon data into 663 1-week bins from 54683.0 to 59324.0
LightCurve: select 655 cells for fitting with e>0.5 & n>2


## Test/Demonstration with 3C 279

> Note that this also demonstrates using `nbdoc` to have a single Jupyterlab cell generate a document

First, the weekly light curve:
{out1}    <figure style="margin-left: 5%" title="Figure 1">  <a href="images/demo_fig_01.png" title="images/demo_fig_01.png">    <img src="images/demo_fig_01.png" alt="Figure 1 at images/demo_fig_01.png" width=600>   </a>  <figcaption><b>Figure 1</b>. Full scale</figcaption></figure>

Replot the figure with an expanded scale to see position of a flare:
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/demo_fig_02.png" title="images/demo_fig_02.png">    <img src="images/demo_fig_02.png" alt="Figure 2 at images/demo_fig_02.png" width=600>   </a> </figure>
<details  class="nbdoc-description" >  <summary> Define orbit-based subset around large flare at MJD 57189 </summary>  <div style="margin-left: 5%"><pre>CellData: Bin photon data into 80 orbit-based bins from 57186.1 to 57191.0<br>LightCurve: select 61 cells for fitting with e&gt;0.5 & n&gt;2<br></pre></div> </details>    <figure style="margin-left: 5%" title="Figure 3">  <a href="images/demo_fig_03.png" title="images/demo_fig_03.png">    <img src="images/demo_fig_03.png" alt="Figure 3 at images/demo_fig_03.png" width=600>   </a> </figure>
<details  class="nbdoc-description" >  <summary> Apply BB and overplot it with the cells on which it is based </summary>  <div style="margin-left: 5%"><pre>LightCurve: select 61 cells for fitting with e&gt;0.5 & n&gt;2<br>Partitioned 61 cells into 10 blocks, using LikelihoodFitness <br>LightCurve: Loaded 10 / 10 cells for fitting<br></pre></div> </details>    <figure style="margin-left: 5%" title="Figure 4">  <a href="images/demo_fig_04.png" title="images/demo_fig_04.png">    <img src="images/demo_fig_04.png" alt="Figure 4 at images/demo_fig_04.png" width=600>   </a> </figure>

Table of BB fits
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>t</th>
      <th>tw</th>
      <th>n</th>
      <th>ts</th>
      <th>flux</th>
      <th>errors</th>
      <th>limit</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>57186.116</td>
      <td>0.017</td>
      <td>3</td>
      <td>12.800</td>
      <td>1.498</td>
      <td>(-0.745, 1.096)</td>
      <td>4.006</td>
    </tr>
    <tr>
      <td>57186.253</td>
      <td>0.025</td>
      <td>11</td>
      <td>19.200</td>
      <td>2.693</td>
      <td>(-0.998, 1.264)</td>
      <td>5.260</td>
    </tr>
    <tr>
      <td>57186.385</td>
      <td>0.026</td>
      <td>10</td>
      <td>21.300</td>
      <td>2.743</td>
      <td>(-0.989, 1.246)</td>
      <td>5.263</td>
    </tr>
    <tr>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <td>57190.831</td>
      <td>0.012</td>
      <td>9</td>
      <td>29.100</td>
      <td>3.833</td>
      <td>(-1.332, 1.695)</td>
      <td>7.289</td>
    </tr>
    <tr>
      <td>57190.898</td>
      <td>0.009</td>
      <td>4</td>
      <td>5.700</td>
      <td>1.792</td>
      <td>(-1.028, 1.527)</td>
      <td>5.314</td>
    </tr>
    <tr>
      <td>57190.966</td>
      <td>0.006</td>
      <td>3</td>
      <td>2.700</td>
      <td>2.005</td>
      <td>(-1.435, 2.221)</td>
      <td>7.306</td>
    </tr>
  </tbody>
</table>
</div>

This can be compared with Figure 4 from the [Kerr paper](https://arxiv.org/pdf/1910.00140.pdf)
<figure style="margin-left: 5%" title="Figure 5">  <a href="images/demo_fig_05.png" title="images/demo_fig_05.png">    <img src="images/demo_fig_05.png" alt="Figure 5 at images/demo_fig_05.png" width=600>   </a> </figure>


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_weights.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90-main.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Mon May 10 17:20:16 PDT 2021
